Instalación de todas las librerías del proyecto.

In [1]:
# !pip install -r ../requirements.txt

In [2]:
import pandas as pd
import numpy as np
import os


In [3]:
if not os.path.isfile("../gun_violence_dataset/gun_violence_dataset.csv"):
    import tarfile

    tar = tarfile.open("../data/gun_violence.tar.gz", "r:gz")
    tar.extractall(path="../gun_violence_dataset")
    tar.close()

    os.rename("../gun_violence_dataset/stage3.csv","../gun_violence_dataset/gun_violence_dataset.csv")


In [4]:
# https://github.com/jamesqo/gun-violence-data
incidents = pd.read_csv('../gun_violence_dataset/gun_violence_dataset.csv')
incidents.head()

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,...,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||3::Male||4::Female,0::Julian Sims,NaN,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://pittsburgh.cbslocal.com/2013/01/01/4-pe...,NaN,NaN
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male,0::Bernard Gillis,NaN,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://losangeles.cbslocal.com/2013/01/01/man-...,62.0,35.0
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,...,0::25||1::31||2::33||3::34||4::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,0::Damien Bell||1::Desmen Noble||2::Herman Sea...,NaN,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...,http://www.morningjournal.com/general-news/201...,56.0,13.0
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,...,0::29||1::33||2::56||3::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Female||1::Male||2::Male||3::Male,0::Stacie Philbrook||1::Christopher Ratliffe||...,NaN,0::Killed||1::Killed||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://denver.cbslocal.com/2013/01/06/officer-...,40.0,28.0
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,...,0::18||1::46||2::14||3::47,0::Adult 18+||1::Adult 18+||2::Teen 12-17||3::...,0::Female||1::Male||2::Male||3::Female,0::Danielle Imani Jameison||1::Maurice Eugene ...,3::Family,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://myfox8.com/2013/01/08/update-mother-sho...,62.0,27.0


Veamos el número de registros vacíos por cada columna:

In [5]:
# 240k filas
incidents.isna().sum()

incident_id                         0
date                                0
state                               0
city_or_county                      0
address                         16497
n_killed                            0
n_injured                           0
incident_url                        0
source_url                        468
incident_url_fields_missing         0
congressional_district          11944
gun_stolen                      99498
gun_type                        99451
incident_characteristics          326
latitude                         7923
location_description           197588
longitude                        7923
n_guns_involved                 99451
notes                           81017
participant_age                 92298
participant_age_group           42119
participant_gender              36362
participant_name               122253
participant_relationship       223903
participant_status              27626
participant_type                24863
sources     

Eliminamos las columnas con mayor cantidad de valores nulos (participant_relationship, location_description), que son prescindibles o contienen información personal que no nos interesa. 

Eliminamos también incident_url_fields_missing que siempre tiene valor False y las columnas que tienen las url de las noticias o de los datos del incidente (incident_url, sources, source_url), ya que no tendrán ninguna utilidad en los tratamientos y estudios posteriores. 

Por último, prescindimos también de las columnas state_house_district y state_senate_district, a favor de congressional_district, que es la que menor número de valores nulos tiene y porque las tres columnas se refieren a lo mismo: la entidad territorial que elige a un solo miembro del congreso.

In [6]:
del incidents["participant_relationship"], incidents["location_description"], incidents["incident_url_fields_missing"], incidents["incident_url"], incidents["sources"], incidents["source_url"], incidents["state_house_district"], incidents["state_senate_district"]
incidents.isna().sum()

incident_id                      0
date                             0
state                            0
city_or_county                   0
address                      16497
n_killed                         0
n_injured                        0
congressional_district       11944
gun_stolen                   99498
gun_type                     99451
incident_characteristics       326
latitude                      7923
longitude                     7923
n_guns_involved              99451
notes                        81017
participant_age              92298
participant_age_group        42119
participant_gender           36362
participant_name            122253
participant_status           27626
participant_type             24863
dtype: int64

A continuación, vamos a crear nuevas columnas que resuman la información contenida en celdas con varios valores.

In [7]:
# Devuelve el conjunto de objectos de una celda
def get_elements_as_list(cell):
#     print(cell)
    if('||' in cell):
        return cell.split('||')
    else:
        return cell.split('|')

# Devuelve el número de elementos
def get_num_elements_of_object(cell):
    elements = get_elements_as_list(cell)
    return len(elements)     

# Devuelve un map donde key=i y value=value
def get_elements(cell):
    res = {}
    elements = get_elements_as_list(cell)
    i = 0
    for element in elements:
#         print(element)
        if '::' in element:
            value = element.split('::')[1]
        else:
            value = element.split(':')[1]
        res[i] = value
        i = i + 1
    return res

# Devuelve los elementos (get_elements) como números
def get_elements_as_numbers(cell):
    res = get_elements(cell)
    
    # Convert to number
    for i in range(0, len(res)):
        res[i] = float(res[i])
        
    return res

# Calcula el valor medio de una columna con objetos de tipo número
def compute_average_value(cell):
    if not pd.isnull(cell):
        elements = get_elements_as_numbers(cell)
        ac = 0.0
        
        for i in range(0, len(elements)):
            ac = ac + elements[i]
            
        return ac/len(elements)
    else:
        return None

# Get number of victims and suspects
def get_number_of_value(cell, value):
    elements = get_elements(cell)
    
    n_values = 0
    
    for i in range(0, len(elements)):
        if(elements[i].lower() == value.lower()):
            n_values = n_values + 1
    
    return n_values

# Returns true if an object contains part of a string
def get_contains(cell, value):
    if not pd.isnull(cell):
        elements = get_elements_as_list(cell)
        
        for i in range(0, len(elements)):
            if value in elements[i].lower():
                return True
        
        return False
    else:
        return None

def get_contains_multiple(cell, values):
    if not pd.isnull(cell):
        res = False
        for value in values:
            res = get_contains(cell, value)
            
            if res:
                return res
        
        return res
    else:
        return None

# Returns true if an object contains exactly a string
def get_contains_strict(cell, value):
    
    elements = get_elements_as_list(cell)
    
    for i in range(0, len(elements)):
        if value == elements[i].lower():
            return True
    
    return False

# Returns true if an object contains exactly one element of a list of strings
def get_contains_strict_multiple(cell, values):
    if not pd.isnull(cell):
        res = False
        for value in values:
            res = get_contains_strict(cell, value)
            
            if res:
                return res
        
        return res
    else:
        return None

# Imprime un valor si este no aparece en la lista pasada por parámetro, se usa para encontrar valores no contemplados en parámetros de tipo enumerado
def print_not_considered(cell, values, simple_list=False):
    
    if simple_list:
        elements = get_elements_as_list(cell)
    else:
        elements = get_elements(cell)
        
#     print(elements)
    
    for i in range(0, len(elements)):
        if elements[i].lower() not in values:
#             print('Values: {}'.format(values))
#             print('Element:{}'.format(elements[i]))
            print(elements[i])

# Similar to print_not_considered, but with substrings
def print_not_considered_contains(cell, values):
    
    elements = get_elements_as_list(cell)
             
            
    for i in range(0, len(elements)):
        contains = False
        for value in values:
            if value in elements[i].lower():
                contains = True
        
        if not contains:
            print(elements[i])

## 1.- Edad media de los participantes

In [8]:
incidents['average_age'] = incidents['participant_age'].apply(compute_average_value)

## 2.- Número de víctimas

In [9]:
# Antes de nada, comprobamos que no existen más valores en la columna participant_type, exceptuando Victim y subject-suspect
def print_not_considered_participant_type(cell):
    if not pd.isnull(cell):
        return print_not_considered(cell, ['victim', 'subject-suspect'])
    else:
        return None

incidents['participant_type'].apply(print_not_considered_participant_type)

def get_num_victims(cell):
    if not pd.isnull(cell):
        return get_number_of_value(cell, 'Victim')
    else:
        return None

incidents['n_victims'] = incidents['participant_type'].apply(get_num_victims)

## 3.- Número de sospechosos 

In [10]:
def get_num_suspects(cell):
    if not pd.isnull(cell):
        return get_number_of_value(cell, 'Subject-Suspect')
    else:
        return None

incidents['n_suspects'] = incidents['participant_type'].apply(get_num_suspects)

## 4.- Convertir gun_stolen a boolean
True si alguna de las armas ha sido robada, False en el caso contrario o Unknown si no se sabe. Borrar esta columna, ya forma parte de incident-type

In [11]:
def print_not_considered_gun_stolen(cell):
    if not pd.isnull(cell):
        if "not-stolen" in cell.lower():
            return False
        else:
            if "stolen" in cell.lower():
                return True
            else:
                return "Unknown"
    else:
        return "Unknown"

incidents["stolen_gun_involved"] = incidents['gun_stolen'].apply(print_not_considered_gun_stolen)

## 5.- Crear boolean minors_involved

In [12]:
def print_not_considered_participant_age_group(cell):
    if not pd.isnull(cell):
        return print_not_considered(cell, ['adult 18+', 'teen 12-17', 'child 0-11'])
    else:
        return None

incidents['participant_age_group'].apply(print_not_considered_participant_age_group)

incidents["minors_involved"] = incidents['participant_age_group'].apply(get_contains_multiple, values=['teen 12-17', 'child 0-11'])

## 6.- Porcentaje de mujeres involucradas

In [13]:
def get_female_percentage(cell):
    if not pd.isnull(cell):
        genders = list(get_elements(cell).values())
        female = genders.count("Female")
        male = genders.count("Male")
        return female/(len(genders))*100

    else:
        return None

incidents["women_percentage"] = incidents['participant_gender'].apply(get_female_percentage)

## 7.- Convertir tipos de incidentes en booleans

In [14]:
def get_incident_types():
    return [
                                    'Domestic Violence'.lower(),
                                    'Drive-by'.lower(),
                                    'Gang involvement'.lower(),
                                    'Home Invasion'.lower(),
                                    'Institution/Group/Business'.lower(),
                                    'Shooting'.lower(),
                                    'Murder'.lower(),
                                    'Officer Involved'.lower(),
                                    'Possession'.lower(),
                                    'Shot'.lower(),
                                    'Suicide'.lower(),
                                    'Drug involvement'.lower(),
                                    'Stolen/Illegally owned gun'.lower(),
                                    'Bar/club incident'.lower(),
                                    'Sex crime'.lower(),
                                    'School Incident'.lower(),
                                    'Kidnapping/abductions/hostage'.lower(),
                                    'Defensive Use'.lower(),
                                    'Car-jacking'.lower(),
                                    'Armed robbery'.lower(),
                                    'Hate crime'.lower(),
                                    'Child'.lower(),
                                    'Accidental/Negligent Discharge'.lower(),
                                    'Brandishing/flourishing/open carry/lost/found'.lower(),
                                    'Gun(s) stolen'.lower(),
                                    'Guns stolen'.lower(), # Group with the previous one
                                    'Road rage'.lower(),
                                    'Under the influence of alcohol or drugs'.lower(),
                                    'BB/Pellet/Replica gun'.lower(),
                                    'Cleaning gun'.lower(),
                                    'Implied Weapon'.lower(),
                                    'House party'.lower(),
                                    'ATF/LE Confiscation/Raid/Arrest'.lower(),
                                    'Gun at school'.lower(),
                                    'Thought gun was unloaded'.lower(),
                                    'Hunting accident'.lower(),
                                    'stolen gun'.lower(),
                                    'Police Targeted'.lower(),
                                    'Pistol-whipping'.lower(),
                                    'Playing with gun'.lower(),
                                    'Shootout'.lower(),
                                    'Unlawful purchase/sale'.lower(),
                                    'Non-Aggression'.lower(),
                                    'Gun shop robbery or burglary'.lower(),
                                    'Concealed Carry License'.lower(),
                                    'Assault weapon'.lower(),
                                    'LOCKDOWN/ALERT'.lower(),
                                    'TSA Action'.lower(),
                                    'Terrorism'.lower(),
                                    'Ghost gun'.lower(),
                                    'Political Violence'.lower(),
                                    'Mistaken ID'.lower(),
                                    'NAV'.lower(),
                                    'Gun buy back action'.lower()
                                ]

def print_not_considered_incident_characteristics(cell):
    if not pd.isnull(cell):
        incident_types = get_incident_types()
        return print_not_considered_contains(cell, 
                                    incident_types
                                )
    else:
        return None

incidents['incident_characteristics'].apply(print_not_considered_incident_characteristics)

for incident_type in get_incident_types():
    incidents[incident_type] = incidents['incident_characteristics'].apply(get_contains, value=incident_type)

Una vez eliminadas las columnas con mayor número de valores faltantes y aquellas que no nos son de interés, y creadas columnas con nueva información, procedemos a dividir el dataset en dos subdatasets, uno de incidentes, sin la información agregada de las personas; y otro de personas; cada uno de los cuales será tratado en cuanto a valores nulos y faltantes, en lugar de hacer esto antes de dividirlo pues se perdería información.

# Dataset de incidentes

En primer lugar vamos a eliminar columnas con valores agregados de personas o armas que ya no necesitamos al haber resumido esa información en nuevas columnas.

In [15]:
processed_incidents = incidents.copy(deep=True)
del processed_incidents["gun_stolen"], processed_incidents["gun_type"], processed_incidents["incident_characteristics"], processed_incidents["participant_age"], processed_incidents["participant_age_group"], processed_incidents["participant_gender"], processed_incidents["participant_name"], processed_incidents["participant_status"], processed_incidents["participant_type"]
processed_incidents.head()

,incident_id,date,state,city_or_county,address,n_killed,n_injured,congressional_district,latitude,longitude,...,concealed carry license,assault weapon,lockdown/alert,tsa action,terrorism,ghost gun,political violence,mistaken id,nav,gun buy back action
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,14.0,40.3467,-79.8559,...,False,False,False,False,False,False,False,False,False,False
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,43.0,33.9090,-118.3330,...,False,False,False,False,False,False,False,False,False,False
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,9.0,41.4455,-82.1377,...,False,False,False,False,False,False,False,False,False,False
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,6.0,39.6518,-104.8020,...,False,False,False,False,False,False,False,False,False,False
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,6.0,36.1140,-79.9569,...,False,False,False,False,False,False,False,False,False,False


Estudiemos ahora el número de valores nulos que manejamos, teniendo en cuenta que tenemos unas 240.000 filas:

In [16]:
pd.set_option('display.max_rows',None)
processed_incidents.isna().sum()

incident_id                                          0
date                                                 0
state                                                0
city_or_county                                       0
address                                          16497
n_killed                                             0
n_injured                                            0
congressional_district                           11944
latitude                                          7923
longitude                                         7923
n_guns_involved                                  99451
notes                                            81017
average_age                                      92298
n_victims                                        24863
n_suspects                                       24863
stolen_gun_involved                                  0
minors_involved                                  42119
women_percentage                                 36362
domestic v

Podemos observar que las columnas con mayor número de valores nulos son n_guns_involved, average_age y notes. El resto, tienen un número de valores nulos asumible. Si prescindiéramos de auqellas filas con algún valor nulo quedaríamos con unas 55.000 filas:

In [17]:
(processed_incidents.dropna()).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55341 entries, 4 to 239676
Data columns (total 72 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   incident_id                                    55341 non-null  int64  
 1   date                                           55341 non-null  object 
 2   state                                          55341 non-null  object 
 3   city_or_county                                 55341 non-null  object 
 4   address                                        55341 non-null  object 
 5   n_killed                                       55341 non-null  int64  
 6   n_injured                                      55341 non-null  int64  
 7   congressional_district                         55341 non-null  float64
 8   latitude                                       55341 non-null  float64
 9   longitude                                      55

Estaríamos perdiendo casi 200.000 registros, lo cual no es óptimo. Preferimos no rellenar los valores faltantes con medias o modas por no introducir ruido y, en principio, con vistas a tener el mayor número de incidentes de cara a los estudios gráficos geográficos que vamos a realizar, no vamos a eliminar los valores nulos existentes, simplemente seremos conscientes de ellos en las distintas tareas que realicemos. Lo que sí vamos a eliminar son los 7.000 incidentes que no tienen latitud y longitud por perder este interés que estamos comentando:

In [18]:
processed_incidents = processed_incidents.dropna(subset=['latitude', "longitude"])

Por último, almacenamos este dataset:

In [19]:
processed_incidents.to_csv("../gun_violence_dataset/incidents_dataset.csv")

# Dataset de personas

Extraemos la información de las personas involucradas en un dataset aparte. Registraremos:
* Name
* Age
* Age_group
* Gender
* Status
* Type
* State
* City_or_county
* N_killed
* N_injured
* Average_age
* N_victims
* N_suspects
* Stolen_gun_involved
* Minors_involved

Generamos el dataset (tarda un poco):

In [23]:
import csv
from tqdm import tqdm
f = open("../gun_violence_dataset/people_dataset.csv", "w", newline='', encoding="UTF-8")
csvwriter = csv.writer(f, delimiter=',')
csvwriter.writerow(['Name', 'Age', 'Age_group', 'Gender', 'Status', 'Type', 'Incident_id', 'State', 'City_or_county', 'N_killed', 'N_injured', "Average_age", "N_victims", "N_suspects", "Stolen_gun_involved", "Minors_involved"])

counter = 0

for index, row in tqdm(incidents.iterrows()):
    average_age = row["average_age"] if not pd.isnull(row["average_age"]) else None
    n_victims = row["n_victims"] if not pd.isnull(row["n_victims"]) else None
    n_suspects = row["n_suspects"] if not pd.isnull(row["n_suspects"]) else None
    stolen_gun_involved = row["stolen_gun_involved"] if not pd.isnull(row["stolen_gun_involved"]) else None
    minors_involved = row["minors_involved"] if not pd.isnull(row["minors_involved"]) else None
    types = get_elements(row["participant_type"]) if not pd.isnull(row["participant_type"]) else {}
    statuses = get_elements(row["participant_status"]) if not pd.isnull(row["participant_status"]) else {}
    genders = get_elements(row["participant_gender"]) if not pd.isnull(row["participant_gender"]) else {}
    age_groups = get_elements(row["participant_age_group"]) if not pd.isnull(row["participant_age_group"]) else {}
    ages = get_elements(row["participant_age"]) if not pd.isnull(row["participant_age"]) else {}
    participant_names = get_elements(row["participant_name"]) if not pd.isnull(row["participant_name"]) else {}

    participants_ids = list(types.keys()) + list(statuses.keys()) + list(genders.keys()) + list(age_groups.keys()) + list(ages.keys()) + list(participant_names.keys())
    participants_ids = list(dict.fromkeys(participants_ids)) # delete duplicates

    for i in participants_ids:
        name = participant_names[i] if i in participant_names.keys() else None
        age = ages[i] if i in ages.keys() else None
        age_group = age_groups[i] if i in age_groups.keys() else None
        gender = genders[i] if i in genders.keys() else None
        status = statuses[i] if i in statuses.keys() else None
        tyype = types[i] if i in types.keys() else None

        csvwriter.writerow([name, age, age_group, gender, status, tyype, row["incident_id"], row["state"], row["city_or_county"], row["n_killed"], row["n_injured"], average_age, n_victims, n_suspects, stolen_gun_involved, minors_involved])

f.close()

239677it [00:41, 5708.69it/s]


Lo recuperamos y vemos el tamaño del mismo:

In [24]:
people = pd.read_csv('../gun_violence_dataset/people_dataset.csv')
people.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392323 entries, 0 to 392322
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Name                 180904 non-null  object 
 1   Age                  221671 non-null  float64
 2   Age_group            336618 non-null  object 
 3   Gender               352263 non-null  object 
 4   Status               379334 non-null  object 
 5   Type                 392323 non-null  object 
 6   Incident_id          392323 non-null  int64  
 7   State                392323 non-null  object 
 8   City_or_county       392323 non-null  object 
 9   N_killed             392323 non-null  int64  
 10  N_injured            392323 non-null  int64  
 11  Average_age          276485 non-null  float64
 12  N_victims            392323 non-null  float64
 13  N_suspects           392323 non-null  float64
 14  Stolen_gun_involved  392323 non-null  object 
 15  Minors_involved  

In [25]:
people.head()

,Name,Age,Age_group,Gender,Status,Type,Incident_id,State,City_or_county,N_killed,N_injured,Average_age,N_victims,N_suspects,Stolen_gun_involved,Minors_involved
0,Julian Sims,20.0,Adult 18+,Male,Arrested,Victim,461105,Pennsylvania,Mckeesport,0,4,20.0,4.0,1.0,Unknown,False
1,NaN,NaN,Adult 18+,Male,Injured,Victim,461105,Pennsylvania,Mckeesport,0,4,20.0,4.0,1.0,Unknown,False
2,NaN,NaN,Adult 18+,Male,Injured,Victim,461105,Pennsylvania,Mckeesport,0,4,20.0,4.0,1.0,Unknown,False
3,NaN,NaN,Adult 18+,Female,Injured,Victim,461105,Pennsylvania,Mckeesport,0,4,20.0,4.0,1.0,Unknown,False
4,NaN,NaN,Adult 18+,NaN,Injured,Subject-Suspect,461105,Pennsylvania,Mckeesport,0,4,20.0,4.0,1.0,Unknown,False


## 1.- Convertir tipo de personas en boolean

In [26]:
# print(people["Type"].unique()) # ['Victim', 'Subject-Suspect']

people = people.join(pd.get_dummies(people["Type"]))
people = people.drop("Type", axis = 1)


## 2.- Convertir estado de la persona en boolean

In [27]:
# print(people["Status"].unique()) 

def delete_spaces(cell):
    if not pd.isnull(cell):
        return cell.replace(" ","")
    else:
        return None

people = people.join(people['Status'].apply(delete_spaces).str.get_dummies(sep=','))
people = people.drop("Status", axis = 1)


## 3.- Tratamiento de valores nulos

Existen alrededor de 400.000 registros en el dataset de personas. Veamos para cada columna cuántos valores vacíos tenemos:

In [28]:
people.isna().sum()

Name                   211419
Age                    170652
Age_group               55705
Gender                  40060
Incident_id                 0
State                       0
City_or_county              0
N_killed                    0
N_injured                   0
Average_age            115838
N_victims                   0
N_suspects                  0
Stolen_gun_involved         0
Minors_involved         29098
Subject-Suspect             0
Victim                      0
Arrested                    0
Injured                     0
Killed                      0
Unharmed                    0
dtype: int64

Podemos observar que, efectivamente, el nombre es la columna con mayor número de valores faltantes, seguido de la edad de la persona y la media de edad del incidente en que se vio involucrada. Tiene sentido que prescindamos del nombre, ya que aporta poco de cara al análisis o estudio del dataset, pero sí nos parece interesante mantener la edad de la persona o la media de edad del incidente. 

Creemos que no es adecuado en este caso rellenar los valores faltantes con alguna técnica como la media o la moda, o incluso con predicción o regresión; ya que son del orden de entre 170.000 y 30.000 para las distintas columnas, por lo que introduciríamos un error muy considerable. Por ello, eliminamos la columna "Name" y el resto de valores vacíos:

In [29]:
people = people.drop("Name", axis = 1)
people = people.dropna()
people.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 217671 entries, 0 to 392322
Data columns (total 19 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Age                  217671 non-null  float64
 1   Age_group            217671 non-null  object 
 2   Gender               217671 non-null  object 
 3   Incident_id          217671 non-null  int64  
 4   State                217671 non-null  object 
 5   City_or_county       217671 non-null  object 
 6   N_killed             217671 non-null  int64  
 7   N_injured            217671 non-null  int64  
 8   Average_age          217671 non-null  float64
 9   N_victims            217671 non-null  float64
 10  N_suspects           217671 non-null  float64
 11  Stolen_gun_involved  217671 non-null  object 
 12  Minors_involved      217671 non-null  object 
 13  Subject-Suspect      217671 non-null  uint8  
 14  Victim               217671 non-null  uint8  
 15  Arrested         

De esta forma, nos quedamos finalmente con 216.000 registros sin ningún valor faltante, lo cual, entendemos, son suficientes para los estudios posteriores. Guardamos el dataset en un archivo:

In [30]:
people.to_csv("../gun_violence_dataset/people_dataset.csv")